In [1]:
from datetime import datetime
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time


In [ ]:


consumer_key = <use-your-key>
consumer_secret = <use-your-key>
access_token = <use-your-key>
access_token_secret = <use-your-key>

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [7]:
username = '_fertileground_'
count = 1000
try:
    
    tweets = tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(count)


    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text, tweet.user.location]
                   for tweet in tweets]

   
    tweets_df = pd.DataFrame(tweets_list)
except BaseException as e:
    print('failed on_status,', str(e))
    time.sleep(3)

tweets_df.columns = ['Tweet Date', 'Tweet ID', 'Tweet text', 'Tweet Location']
print(tweets_df)
#tweets_df.to_csv('fertilegroundtweets.csv')

             Tweet Date             Tweet ID  \
0   2021-03-18 18:21:57  1372614266386329600   
1   2021-03-18 02:28:27  1372374308786466824   
2   2021-03-16 15:59:11  1371853563475988482   
3   2021-03-11 23:04:26  1370148640539430919   
4   2021-03-11 23:02:15  1370148088678060037   
..                  ...                  ...   
562 2019-06-19 21:08:20  1141452680130420738   
563 2019-06-19 21:04:46  1141451780070477826   
564 2019-06-19 20:49:10  1141447855363829760   
565 2019-03-21 21:07:00  1108837436958953473   
566 2019-02-02 16:49:42  1091740454146723841   

                                            Tweet text Tweet Location  
0    RT @NFUDC: When restaurants closed during the ...    Jackson, MS  
1    RT @cadwego: “Even as someone who is fairly cr...    Jackson, MS  
2    RT @BloombergDotOrg: We believe in the power o...    Jackson, MS  
3    Thank you for supporting our local community i...    Jackson, MS  
4    RT @MikeBloomberg: Many would like to forget t...    Jacks

In [15]:
#for different search terms and to get the reply count, retweet counts and other data from the tweets related to the search term.

import snscrape.modules.twitter as sntwitter


tweets_list2 = []



terms=['fertile ground', 'food access', 'food insecurity', 'food security', 'food health', 'food deserts', 'food swamps','#fertilegroundjxn','#healthyfood']
geocode = '32.2998,-90.1848'
radius = '50km'
date = "2018-03-01"

for term in terms:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):
   
        if i>500:
            break
      
        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])
    
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])
tweets_df2.drop_duplicates(subset="Tweet Id", keep=False, inplace=True)
tweets_df2
#tweets_df2.to_csv("tweetsallCounts.csv")

,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2021-01-22 04:12:08+00:00,1352469068452868103,@GanucheauAdam Thank you for retweeting! Also ...,_fertileground_,"Jackson, MS",0,0,0,0
1,2021-01-22 01:00:14+00:00,1352420775530291202,We’ve BEEN asking for this!\n\nKey House leade...,_fertileground_,"Jackson, MS",0,0,0,0
2,2020-11-30 22:02:50+00:00,1333531965245566977,Mississippi Goals! https://t.co/DrQE0Ycn3j,_fertileground_,"Jackson, MS",0,0,2,0
3,2020-11-30 22:01:50+00:00,1333531710907150336,Tips! https://t.co/iyte1eETRN,_fertileground_,"Jackson, MS",0,0,0,0
4,2020-11-18 12:44:34+00:00,1329042818199908353,@JacksonStateU Athletics and @UHC will be prov...,_fertileground_,"Jackson, MS",0,0,0,0
...,...,...,...,...,...,...,...,...,...
152,2018-07-17 18:54:06+00:00,1019294187592802309,#transformationtuesday #newbody #healthylifest...,epicdanceandfit,"Ridgeland, MS",0,0,1,0
153,2018-07-14 02:40:19+00:00,1017961962251186177,#postrunfuel #berries #strawberries #strawberr...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0
154,2018-07-11 02:06:15+00:00,1016866225324949505,#postrunmeal #crabsalad #and #sweetpotato #org...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0
155,2018-06-27 02:36:25+00:00,1011800386062209025,#postrunmeal #healthyfood #foodporn #foodporno...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0


In [19]:
#for demo
import snscrape.modules.twitter as sntwitter


tweets_list2 = []


#terms=['fertile ground', 'food access', 'food insecurity', 'food security', 'food health', 'food deserts', 'food swamps','#fertilegroundjxn','#healthyfood']
term='food'
geocode = '32.2998,-90.1848'
radius = '50km'
date = "2018-03-01"


for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):
   
    if i>50:
        break
      
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])
    
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])
tweets_df2.drop_duplicates(subset="Tweet Id", keep=False, inplace=True)

#tweets_df2.to_csv("food.csv")
tweets_df2

,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2021-04-22 02:26:19+00:00,1385057347291189258,They gone to get food I’m calling front desk A...,nekoty7,"Jackson, MS",0,0,0,0
1,2021-04-22 02:19:07+00:00,1385055536325005312,"@BrandonPresley That’s a lot of food, Commissi...",Jim_Warren_,"Jackson, Mississippi",0,0,1,0
2,2021-04-22 01:49:07+00:00,1385047988180070413,'Inspiring': 8-year-old boy holds food drives ...,16WAPTNews,"Jackson, MS",0,4,13,0
3,2021-04-22 01:16:07+00:00,1385039679414800387,Rushing to pick up your food while Warriors ar...,HakeemDaXtreme,"Jackson, MS",0,0,0,0
4,2021-04-22 01:14:25+00:00,1385039254435270663,@ChefGruel @bdomenech And keep up the food porn!,chefmaw,"Madison, MS",0,0,1,0
5,2021-04-22 01:14:00+00:00,1385039149644750848,MPB FOUNDATION'S LIVE VIRTUAL EVENT: You're in...,MPBOnline,"Jackson, MS",0,0,1,0
6,2021-04-21 23:46:55+00:00,1385017235589287940,@TeamPulte Paid for a lady's prescriptions thi...,LuckyRiego007,"Florence, MS",0,0,3,0
7,2021-04-21 23:30:07+00:00,1385013004505391109,Amazon debuts pay-by-palm at some Whole Foods ...,WJTV,"Jackson, MS",0,0,0,0
8,2021-04-21 23:09:48+00:00,1385007892168855553,"Today, your Department of Agriculture &amp; Co...",MSDeptofAg,"Jackson, MS",0,5,8,0
9,2021-04-21 22:33:27+00:00,1384998745914421249,@all_in_yo_girl These food tic too people gott...,kidhippie_,"Ridgeland, MS",0,1,1,0


In [26]:
#for individual different search terms and to get the reply count, retweet counts and other data from the tweets related to the search term.

import pandas as pd
import snscrape.modules.twitter as sntwitter

def difkeyword(keyword):
    tweets_list2 = []



    #terms=['fertile ground', 'food access', 'food insecurity', 'food security', 'food health', 'food deserts', 'food swamps','#fertilegroundjxn','#healthyfood']
    #term='#healthyfood'
    term=keyword
    geocode='32.2998,-90.1848'
    radius='50km'
    date="2018-03-01"

    #for term in terms:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):

        if i>500:
            break

        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])




    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])



   # tweets_df2.to_csv("tweets_healthyfood.csv")
    return tweets_df2
    
tweets_df2=difkeyword('fertile ground')
tweets_df2

,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2021-01-22 04:12:08+00:00,1352469068452868103,@GanucheauAdam Thank you for retweeting! Also ...,_fertileground_,"Jackson, MS",0,0,0,0
1,2021-01-22 01:00:14+00:00,1352420775530291202,We’ve BEEN asking for this!\n\nKey House leade...,_fertileground_,"Jackson, MS",0,0,0,0
2,2020-11-30 22:02:50+00:00,1333531965245566977,Mississippi Goals! https://t.co/DrQE0Ycn3j,_fertileground_,"Jackson, MS",0,0,2,0
3,2020-11-30 22:01:50+00:00,1333531710907150336,Tips! https://t.co/iyte1eETRN,_fertileground_,"Jackson, MS",0,0,0,0
4,2020-11-18 12:44:34+00:00,1329042818199908353,@JacksonStateU Athletics and @UHC will be prov...,_fertileground_,"Jackson, MS",0,0,0,0
5,2020-11-16 23:15:57+00:00,1328476931952676864,Exciting! https://t.co/fPRSbRevlz,_fertileground_,"Jackson, MS",0,0,0,0
6,2020-11-12 18:21:17+00:00,1326953227703693313,Congrats to the team behind the Galloway Livin...,_fertileground_,"Jackson, MS",0,1,3,0
7,2020-10-30 13:56:36+00:00,1322175576149499904,Check out our latest podcast with @GovLovePodc...,_salamrida,"Jackson, MS",0,7,5,0
8,2020-09-21 18:46:17+00:00,1308115349242548232,Join us tonight to learn how to make your own ...,_fertileground_,"Jackson, MS",0,0,0,0
9,2020-09-01 18:01:36+00:00,1300856347265830912,Yesterday's behind the scene shooting of the u...,MrDJBaker,Oklahoma,0,1,3,0


In [ ]:
#For news data 
#getting time-out exception now due to issues with API, but I have already collected the data that I needed. 
#check out datascrape_old.ipynb file for sample output

from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd
from newspaper import Article
from newspaper import Config
import nltk


googlenews=GoogleNews(start='03/01/2018',end='03/21/2021')
googlenews.search('food insecurity Mississippi')
result=googlenews.result()
df=pd.DataFrame(result)

for i in range(2,15):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 50

list=[]
for ind in df.index:
    try:
        dict={}
        article = Article(df['link'][ind],config=config)
        article.download()
        article.parse()
        article.nlp()
        dict['Date']=df['date'][ind]
        dict['Media']=df['media'][ind]
        dict['Title']=article.title
        dict['Article']=article.text
        dict['Summary']=article.summary
        list.append(dict)
    except:
        print("caught a timeout")
news_df=pd.DataFrame(list)
googlenews.clear()

news_df